In [1]:
# read file 

data = open("Input/data.txt", "r")
param = open("Input/parameters.txt", "r")



In [2]:
# read lines from file as numbers   

rainfall = []
for line in data:
    rainfall.append(float(line))


N = int(param.readline())

trans_matrix = []

for i in range(N):
    # read line from param
    p = [float(x) for x in param.readline().split()]
    trans_matrix.append(p)


for i in range(N):
    

    




    


[[0.7, 0.3], [0.1, 0.9]]